# **Source Code Analysis (chatwithcode):**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os

ROOT = "/content/drive/MyDrive/Generative AI/End-to-End Projects/1. Source Code Analysis"
os.chdir(ROOT)

!pwd

/content/drive/MyDrive/Generative AI/End-to-End Projects/1. Source Code Analysis


## **Install necessary libaries:**

In [2]:
!pip -q install langchain

!pip -q install pinecone-client==2.2.4
!pip -q install unstructured
!pip -q install chromadb

!pip -q install pypdf

!pip -q install sentence-transformers==2.2.2
!pip -q install langchain_google_genai

!pip -q install GitPython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

## **Load necessary Libaries:**

In [3]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.embeddings import HuggingFaceEmbeddings

import pinecone
from langchain.vectorstores import Pinecone
from langchain.vectorstores import Chroma

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.llms import HuggingFaceEndpoint

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain

from langchain.memory import ConversationSummaryMemory, ConversationBufferMemory, ConversationBufferWindowMemory
from langchain.chains import ConversationalRetrievalChain, RetrievalQA, ConversationChain
from tqdm.autonotebook import tqdm

from IPython.display import display, Markdown

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


**Note ---> GenericLoader:**<br>

* **GenericLoader:** In LangChain, the GenericLoader class (langchain.document_loaders.generic.GenericLoader) provides a flexible way to load documents from various sources and prepare them for processing. It acts as a bridge between a data source (like a file system) and LangChain's document processing pipeline.

  * **Keys:**
    *  It allows you to combine an arbitrary blob_loader (responsible for fetching the raw data) with a blob_parser (responsible for parsing the data into a LangChain-compatible format). This provides a high degree of customization for handling different data sources and formats.

    * The documents are loaded lazily, meaning they are only fetched when needed during the processing pipeline. This can be memory-efficient, especially when dealing with large datasets.

      * **Flexibility:** It adapts to diverse data sources and formats through its ability to combine different loaders and parsers.

      * **Efficiency:** Lazy loading optimizes memory usage for large datasets.

      * **Customization:** You can tailor the loading and parsing process to your specific data characteristics.
    
    * The choice between **GenericLoader** and **built-in** loaders depends on your requirements:
      * **Data Source and Format:** <u>If your data comes from a non-standard source or format not covered by built-in loaders, GenericLoader is the way to go.</u>
    
      * **Customization Needs:** <u>If you need fine-grained control over loading and parsing logic, GenericLoader offers more flexibility.</u>

**Note ---> LanguageParser:**<br>

The **LanguageParser** class from langchain.document_loaders.parsers in LangChain is specifically designed to **parse code** files based on their programming language syntax. It aims to improve the performance of question answering models by providing them with more contextually relevant information.


* **Purpose:**

  * **Splits Code into Meaningful Segments:** The LanguageParser analyzes the code file and splits it into smaller units like individual functions, classes, or other top-level code blocks defined within the file. This segmentation helps question answering models focus on specific parts of the code relevant to the user's query.

  * **Improved Accuracy for Code-Related Tasks:** By providing a more granular understanding of the code structure, LanguageParser can potentially lead to better performance when working with question answering tasks that involve code comprehension (e.g., "What does this function do?" or "Where is this variable defined?").


* **Benefits:**

  * **Enhanced Context for Question Answering Models:** LLMs can process smaller, segmented code sections more efficiently, leading to a better understanding of the code's functionality and potentially more accurate answers.

  * **Increased Efficiency:** Breaking down the code into smaller chunks can improve memory usage and processing speed for LLMs compared to feeding them with the entire code file at once.


* **Key Features:**

 * **Language Inference:** The LanguageParser can often infer the programming language used in the code file based on filename extensions (e.g., .py for Python, .java for Java). However, you can also explicitly specify the language if needed.

 * **Customizable Threshold:** You can configure the parser_threshold parameter to control the minimum number of lines required for a code block to be considered for segmentation. This allows you to adjust the granularity of the parsing based on your specific needs.


* **Use Cases:**
 * Building code-focused question answering systems or chatbots that can answer questions about the functionality of code within a specific programming language.

 * Improving the performance of LLMs when dealing with code analysis tasks that require understanding code structure and relationships between different code blocks.









**Note ---> RecursiveCharacterTextSplitter:**<br>
In LangChain, RecursiveCharacterTextSplitter from the langchain.text_splitter module is a text splitter <u>specifically designed to handle large text documents by splitting them into smaller, manageable chunks based on character-level analysis.</u> Here's a breakdown of its functionality and advantages:

* **Functionality:**

  *  **Recursive Approach:** It employs a recursive strategy, meaning it starts by attempting to split the text into chunks based on **larger separators** (e.g., double newline characters for paragraph breaks, newlines for sentences).

  * **Iterative Refinement:** If the resulting chunks are still larger than a specified size (defined by the chunk_size parameter during initialization), it iteratively attempts to split them further using smaller separators (e.g., single whitespace characters for words, individual characters).

  * **Flexibility:** This approach allows you to control the granularity of the split text based on your needs.


* **Advantages:**

  * **Suitable for Large Documents:** It's particularly well-suited for breaking down long documents into manageable pieces for tasks like document summarization or question answering, where context is crucial.

  * **Preserves Sentence and Paragraph Structure:** By starting with larger separators, it often retains the structure of sentences and paragraphs within the resulting chunks, which can be beneficial for tasks that rely on understanding sentence-level or paragraph-level meaning.

  * **Efficiency:** While recursive, it's generally efficient as it stops splitting once the chunks reach the desired size.


* **Alternative Text Splitters in LangChain:** While RecursiveCharacterTextSplitter is a versatile choice, LangChain offers other text splitters depending on your needs:

 * **CharacterTextSplitter:** Splits text into individual characters (useful for specific character-level analysis tasks).

 * **TokenTextSplitter:** Splits text based on whitespace or punctuation, resulting in tokens (words).

 * **Language-Specific Text Splitters:** LangChain provides several built-in text splitters for specific languages (e.g., KonlpyTextSplitter for Korean).


* **Choosing the Right Text Splitter:** The best text splitter for your project depends on your specific application. Consider factors like:

  * **Text Length:** If working with very large documents, RecursiveCharacterTextSplitter might be ideal.

  * **Desired Granularity:** Do you need to split by words, sentences, or individual characters?

  * **Language Considerations:** Are you dealing with a language requiring special treatment?



**Larger Separator:**<br>
In the context of text splitting, "larger separators" refer to punctuation marks or formatting cues that typically indicate boundaries between larger chunks of text, like sentences or paragraphs. Here are some examples of larger separators:

* **Common Larger Separators:**

  * **Newline Characters (\n):** Usually signify the end of a line and often mark the end of a sentence or the start of a new one.

  * **Double Newline Characters (\n\n):** Often indicate the end of a paragraph or a significant break between sections within the text.

  * **Tab Characters (\t):** Though less frequent, tabs can sometimes be used to separate paragraphs or list items.

  * **Sentence-Ending Punctuation:** Punctuation marks like full stops (periods .) or question marks **(?)** can indicate the end of a sentence.


* **Smaller Separators:** If larger separators haven't created chunks of the desired size, the splitter might resort to splitting on smaller separators like single whitespace characters (separating words) or punctuation marks within sentences (e.g., commas or semicolons).


* **Why Use Larger Separators First?** When using a text splitter like RecursiveCharacterTextSplitter in LangChain, the strategy involves splitting text based on these larger separators first. Here's why this approach has advantages:

  * **Preserves Structure:** By prioritizing larger separators, the text splitter maintains the natural structure of the document, keeping sentences and paragraphs intact within the resulting chunks. This can be beneficial for tasks that rely on understanding the relationships between sentences or the overall flow of ideas within paragraphs.

  * **Efficiency:** Splitting based on larger separators is usually more efficient than splitting on individual characters. By initially focusing on these markers, the algorithm can often achieve the desired chunk size quickly.


**Note ---> Language:**<br>
CodeTextSplitter allows you to split your code with multiple languages supported.<br>

```python
    from langchain_text_splitters import Language

    # Full list of supported languages
    lan = [e.value for e in Language]
    print(lan)
    # here is al the language
    # ['cpp', 'go', 'java', 'kotlin', 'js', 'ts', 'php', 'proto', 'python',
    #  'rst', 'ruby', 'rust', 'scala', 'swift',  'markdown', 'latex', 'html',
    #  'sol', 'csharp', 'cobol', 'c', 'lua', 'perl', 'haskell']

```


## **Store a Git Repository:**

In [7]:
!rm -rf test_repo # delete if previously exists

In [8]:
!mkdir test_repo # make a directory for storing the git repositry

In [9]:
# Clone the repo and store it:

repo_path = "test_repo/" # mention the directory name where you want to store the git repo.
Repo.clone_from("https://github.com/dibyendubiswas1998/Document-Tagging.git", to_path=repo_path)

<git.repo.base.Repo '/content/drive/MyDrive/Generative AI/End-to-End Projects/1. Source Code Analysis/test_repo/.git'>

In [10]:
repo_path = "test_repo/"

loader = GenericLoader.from_filesystem(repo_path,
                                        glob = "**/*",
                                       suffixes=[".py"],
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
) # these help to load the specific .py file

document = loader.load() # now load the document, here codebase.

In [ ]:
document

In [12]:
len(document)

24

## **Create the Chunks:**

In [39]:
# Split the documents into multiple chunks, where language is Python:

documents_splitter = RecursiveCharacterTextSplitter.from_language(
    language = Language.PYTHON,
    chunk_size = 2500,
    chunk_overlap = 500
)

chunks = documents_splitter.split_documents(document)

In [40]:
chunks[0]

Document(page_content='from src.document_tagging.pipeline.prediction import PredictionPipeline\nfrom flask import Flask, render_template, request\n\n\n\napp = Flask(__name__)\n\n@app.route("/")\ndef Home():\n    """\n        Renders the "index.html" template for the root URL ("/") in a Flask application.\n\n        Returns:\n            str: The rendered "index.html" template.\n    """\n    return render_template(\'index.html\')\n\n\n@app.route("/prediction", methods=[\'POST\', \'GET\'])\ndef Prediction():\n    """\n        Handles the POST and GET requests for the "/prediction" route.\n        \n        Returns:\n        - The rendered "index.html" template with the predicted tags as a parameter.\n    """\n    pp = PredictionPipeline()\n    tags = None\n    if request.method == \'POST\':\n        messages = request.form[\'messages\']\n        tags = pp.prediction(data=messages) # get all the tags\n    return render_template("index.html", tags=tags)\n\n\n\n\nif __name__ == \'__main__\'

In [41]:
# get the length of chunks:

len(chunks)

59

## **Load the Embedding Model:**

In [4]:
from google.colab import userdata
HUGGINGFACE_API_Token = userdata.get('HuggingFace_API_Token_READ')


In [5]:
# Load the embedding from HuggingFace:

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [19]:
# Check the length of embeddings:

query_result = embeddings.embed_query("Hello I am Dibyendu Biswas")
len(query_result)

384

## **Knowledge Base (Vector DB):**

In [45]:
!rm -rf chromaDB1 # remove chromaDB directory

In [46]:
!mkdir chromaDB1 # create directory to the vector database:

In [ ]:
# Store Embedding into ChromaDB:

persist_directory = 'chromaDB1'
chroma_vector_db = Chroma.from_documents(documents=chunks,
                                 embedding=embeddings,
                                 persist_directory=persist_directory)


In [7]:
# persiste the db to disk:

persist_directory = 'chromaDB1'
# chroma_vector_db.persist()

vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
print(vectordb._collection.count())

59


In [ ]:
# Retrieve the information:

retriever = vectordb.as_retriever(search_kwargs={"k": 15})
docs = retriever.get_relevant_documents("what is Data Ingestion?")
docs

## **Use LLM:**

**Gemini API**

In [8]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY') # get the Gemini API Key



In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GEMINI_API_KEY, temperature=0.4, max_length=512) # load the gemini-pro model
llm

In [ ]:
llm.invoke("Who are you?")

## **Write PromptTemplate for Generating Answer:**

### **Method-01:**

In [12]:
qa_template = """
Use the following information from the context (separated with <ctx></ctx>) to answer the question.
If you don't know the answer, answer with "Unfortunately, I don't have the information." \
If you don't find enough information below, also answer with "Unfortunately, I don't have enough information." \
------
<ctx>
{context}
</ctx>
------
<hs>
{chat_history}
</hs>
------
{question}
Helpful Answer:
"""
qa_prompt = PromptTemplate(template=qa_template, input_variables=['context', 'chat_history', 'question'])

custom_summary_prompt='''
Generate the overall summary of the following text (within the 512 words) that includes the following below elements:

* A title that accurately reflects the content of the text.
* An introduction paragraph that provides an overview of the topic.
* Bullet points that list the key points of the text.
* A conclusion paragraph that summarizes the main points of the text.

Text:`{context}`
'''
custom_summary_template = PromptTemplate(template=custom_summary_prompt, input_variables=['context'])


# define memory:
memory = ConversationBufferWindowMemory(
    llm=llm,
    memory_key="chat_history",
    input_key="question",
    k=5
)


chain_type_kwargs={
        # "verbose": True,
        "question_prompt": qa_prompt,
        "combine_prompt": custom_summary_template,
        "combine_document_variable_name": "context",
        "memory": memory
}

qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="map_reduce",
                                #  return_source_documents=True,
                                 chain_type_kwargs=chain_type_kwargs,
                                 retriever=vectordb.as_retriever(search_kwargs={"k": 15})
                                 )




In [13]:
def generate_response(question, qa_chain=qa_chain):
  ans = qa_chain.invoke(question)
  return display(Markdown(ans['result']))

In [14]:
generate_response(question="Please Explain the code")

**Title: Comprehensive Overview of Machine Learning Code Snippets**

**Introduction:**
This document provides a comprehensive summary of various code snippets related to machine learning tasks, including data preprocessing, model training, evaluation, and prediction. Each section focuses on a specific aspect of the machine learning workflow, providing insights into the code's functionality and purpose.

**Key Points:**

* **Custom Dataset Class:** The `CustomDataset` class inherits from the `Dataset` class and is used to create datasets for training, testing, and validation. It initializes with input IDs, attention masks, and labels, and provides methods to access individual items.

* **Model Evaluation:** The `ModelEvaluation` class evaluates token classification models using test and validation datasets. It computes evaluation metrics like precision, recall, and F1 score, and logs the results into MLflow.

* **Data Preprocessing Configuration:** The `DataPreprocessingConfig` dataclass configures data preprocessing for machine learning tasks. It defines attributes like data file paths, column names, and model parameters, ensuring immutable configuration.

* **Model Training:** The `ModelTraining` class loads pre-trained token classification models, collects training data, and trains the model using the Hugging Face Transformers library.

* **Prediction Pipeline:** The `PredictionPipeline` class is used to make predictions on text data. It instantiates a pipeline that applies word representation techniques and returns tokenized text data, attention masks, and tag data.

* **Word Representation:** The `word_representation` function applies word representation techniques to input text data, converting it into tokenized text data, input IDs, and attention masks. It also logs messages to a specified log file.

* **Model Evaluation Configuration:** The `ModelEvaluationConfig` class represents the configuration for model evaluation. It defines paths to training, test, and validation data, model and tokenizer paths, and other parameters for evaluation.

* **Loading Torch Data:** The `load_torch_data` function loads data from a file using the `torch.load` function. It handles exceptions and returns the loaded data.

**Conclusion:**
These code snippets provide a foundation for understanding various aspects of machine learning workflows. They cover data preprocessing, model training, evaluation, prediction, and configuration, enabling developers to effectively implement machine learning solutions.

In [15]:

generate_response(question="Tell me something about yourself")

**Title:** Insufficient Information: A Persistent Obstacle

**Introduction:**

This text presents a series of statements expressing the lack of sufficient information to address specific inquiries. The repeated use of the phrase "Unfortunately, I don't have the information" highlights the consistent inability to provide meaningful responses.

**Key Points:**

* The text consistently acknowledges a lack of information, indicating an inability to fulfill requests.
* The absence of specific details or context prevents the provision of answers or insights.
* The text does not provide any information or knowledge that could be summarized or analyzed.

**Conclusion:**

The text emphasizes the recurring theme of insufficient information, which serves as a barrier to providing meaningful responses. Without access to the necessary data or knowledge, it is impossible to address the inquiries effectively. The lack of information leaves the text unable to fulfill its intended purpose of providing answers or insights.

In [16]:
memory

ConversationBufferWindowMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Please Explain the code'), AIMessage(content="**Title: Comprehensive Overview of Machine Learning Code Snippets**\n\n**Introduction:**\nThis document provides a comprehensive summary of various code snippets related to machine learning tasks, including data preprocessing, model training, evaluation, and prediction. Each section focuses on a specific aspect of the machine learning workflow, providing insights into the code's functionality and purpose.\n\n**Key Points:**\n\n* **Custom Dataset Class:** The `CustomDataset` class inherits from the `Dataset` class and is used to create datasets for training, testing, and validation. It initializes with input IDs, attention masks, and labels, and provides methods to access individual items.\n\n* **Model Evaluation:** The `ModelEvaluation` class evaluates token classification models using test and validation datasets. It computes evaluation metrics li

### **Method-02:**

In [113]:
qa_template = """
Use the following information from the context (separated with <ctx></ctx>) to answer the question.
If you don't know the answer, answer with "Unfortunately, I don't have the information." \
If you don't find enough information below, also answer with "Unfortunately, I don't have enough information." \
------
<ctx>
{context}
</ctx>
------
<hs>
{chat_history}
</hs>
------
{question}
Helpful Answer:
"""
qa_prompt_2 = PromptTemplate(template=qa_template, input_variables=['context', 'chat_history', 'question'])
memory_2 = ConversationBufferWindowMemory(memory_key="chat_history", input_key="question", k=5)

qa_chain_2 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=vectordb.as_retriever(search_kwargs={"k": 15}),
    # verbose=True,
    chain_type_kwargs={
        # "verbose": True,
        "prompt": qa_prompt_2,
        "memory": memory_2
    }
)

In [149]:
# qa_chain_2.invoke("Who are you?")
generate_response(qa_chain=qa_chain_2, question="Who are you?")

Unfortunately, I don't have the information to answer this question.

In [150]:
# qa_chain_2.invoke("Tell me something about the project.")

generate_response(qa_chain=qa_chain_2, question="Tell me something about the project.")

The project is named "Document Tagging" and it uses the template name "document_tagging". The project structure includes various directories and files for different purposes, such as data storage, model training, logging, and configuration. The project also utilizes various Python libraries and modules for data manipulation, model creation, and evaluation.

The project involves training a token classification model using the Hugging Face Transformers library. The model is trained on a dataset of documents and their corresponding tags. The trained model can be used to tag new documents with the appropriate tags.

The project also includes a pipeline for data preprocessing, model training, model evaluation, and prediction. The pipeline is designed to automate the process of training and deploying the model.

Overall, the project provides a comprehensive framework for training and deploying a token classification model for document tagging tasks.

In [122]:
memory_2

ConversationBufferWindowMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Who are you?'), AIMessage(content='I am a large language model, trained by Google.'), HumanMessage(content='Tell me something about the project.'), AIMessage(content='The project is named "Document Tagging" and it uses the template name "document_tagging". The project structure includes various directories and files for different purposes, such as data storage, model training, logging, and configuration. The project also utilizes various Python libraries and modules for data manipulation, model creation, and evaluation.\n\nThe project involves training a token classification model using the Hugging Face Transformers library. The model is trained on a dataset of documents and their corresponding tags. The trained model can be used to tag new documents with the appropriate tags.\n\nThe project also includes a pipeline for data preprocessing, model training, model evaluation, and prediction. The 